In [6]:
import pathlib
import pandas as pd
import numpy as np
import pingouin as pg
from scipy.stats import ttest_rel
from statsmodels.stats.anova import AnovaRM

ROOT   = pathlib.Path.cwd().parent          # D:\COGBCI
RAWDIR = ROOT / 'results' / 'features_all.csv' 
# 1. 載入特徵資料
df = pd.read_csv(RAWDIR)  # 修改為你的檔案路徑

# 2. 將 cond 轉為數值 load（0b→0, 1b→1, 2b→2）
df['load'] = df['cond'].map({'0b': 0, '1b': 1, '2b': 2})

# 3. 先把每個受試在每個 load 下的 P300 與 Theta 取 trial 平均
agg = df.groupby(['sub', 'load']).agg({
    'p300_mean': 'mean',
    'theta_mean': 'mean'
}).reset_index()

In [11]:
# 4A. 使用 pingouin 做重複量數 ANOVA（P300）
aov_p300 = pg.rm_anova(dv='p300_mean', within='load', subject='sub', data=agg, detailed=True)
print("=== P300 Repeated Measures ANOVA (pingouin) ===")
print(aov_p300, "\n")

# 4B. 使用 pingouin 做成對 t 檢驗（P300），並 Bonferroni 校正
pt_p300 = pg.pairwise_tests(
    dv='p300_mean', within='load', subject='sub',
    data=agg, padjust='bonf', parametric=True
)
print("=== P300 Pairwise Tests (pingouin) ===")
print("Available columns:", list(pt_p300.columns))
print(pt_p300[['A', 'B', 'T', 'p-unc', 'p-corr']].to_string(index=False), "\n")

=== P300 Repeated Measures ANOVA (pingouin) ===
  Source            SS  DF            MS         F    p-unc       ng2  \
0   load  2.359265e-12   2  1.179633e-12  9.881975  0.00035  0.092331   
1  Error  4.536142e-12  38  1.193722e-13       NaN      NaN       NaN   

        eps  
0  0.755375  
1       NaN   

=== P300 Pairwise Tests (pingouin) ===
Available columns: ['Contrast', 'A', 'B', 'Paired', 'Parametric', 'T', 'dof', 'alternative', 'p-unc', 'p-corr', 'p-adjust', 'BF10', 'hedges']
A B         T    p-unc   p-corr
0 1 -1.146525 0.265812 0.797437
0 2  2.801504 0.011386 0.034159
1 2  4.078186 0.000641 0.001923 



C:\Users\tp6mo\anaconda3\envs\bci_env\Lib\site-packages\pingouin\distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d


In [12]:
# 5A. 使用 pingouin 做重複量數 ANOVA（Theta）
aov_theta = pg.rm_anova(dv='theta_mean', within='load', subject='sub', data=agg, detailed=True)
print("=== Theta Repeated Measures ANOVA (pingouin) ===")
print(aov_theta, "\n")

# 5B. 使用 pingouin 的 pairwise_tests（Theta）
pt_theta = pg.pairwise_tests(
    dv='theta_mean', within='load', subject='sub',
    data=agg, padjust='bonf', parametric=True
)
print("=== Theta Pairwise Tests (pingouin) ===")
print("Available columns:", list(pt_theta.columns))
print(pt_theta[['A', 'B', 'T', 'p-unc', 'p-corr']].to_string(index=False), "\n")

=== Theta Repeated Measures ANOVA (pingouin) ===
  Source            SS  DF            MS         F     p-unc      ng2  \
0   load  2.755785e-26   2  1.377892e-26  0.290761  0.749343  0.00142   
1  Error  1.800787e-24  38  4.738913e-26       NaN       NaN      NaN   

        eps  
0  0.838042  
1       NaN   

=== Theta Pairwise Tests (pingouin) ===
Available columns: ['Contrast', 'A', 'B', 'Paired', 'Parametric', 'T', 'dof', 'alternative', 'p-unc', 'p-corr', 'p-adjust', 'BF10', 'hedges']
A B         T    p-unc  p-corr
0 1 -0.930304 0.363884     1.0
0 2 -0.257787 0.799343     1.0
1 2  0.470004 0.643702     1.0 



C:\Users\tp6mo\anaconda3\envs\bci_env\Lib\site-packages\pingouin\distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d
